In [1]:
import torch
import torch.nn as nn
import os
import numpy as np
from collections import OrderedDict
import torch.nn.functional as F
from torch.autograd import Variable
import torchvision.transforms as transforms
from torch.utils.data import DataLoader
from torchvision.datasets.mnist import MNIST
import torch.optim as optim
import cv2
import scipy.misc
from torch.utils.data import Dataset
from skimage import io, transform

In [2]:
class LeNet(nn.Module):
    
    def __init__(self):
        super(LeNet, self).__init__()
        
        self.conv1 = nn.Conv2d(3, 6, 5)
        self.conv2 = nn.Conv2d(6, 16, 5)
        
        n_size = self.get_shape((3, 81, 81))
        
        self.fc1 = nn.Linear(n_size, 120)
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 4)
        
    def get_shape(self, input_shape):
        bs = 1
        input_tensor = Variable(torch.rand(bs, *input_shape))
        output_feat = self.forward_feature(input_tensor)
        n_size = output_feat.data.view(bs, -1).size(1)
        return n_size
    
    def forward_feature(self, x):
        out = F.relu(self.conv1(x))
        out = F.max_pool2d(out, 2)
        out = F.relu(self.conv2(out))
        out = F.max_pool2d(out, 2)
        return out
    
    def forward(self, x):
        out = F.relu(self.conv1(x))
        out = F.max_pool2d(out, 2)
        out = F.relu(self.conv2(out))
        out = F.max_pool2d(out, 2)
        out = out.view(out.size(0), -1)
        out = F.relu(self.fc1(out))
        out = F.relu(self.fc2(out))
        out = F.softmax(self.fc3(out))
        return out

In [3]:
# model = LeNet()
# print(model)

In [4]:
class CellsDataset(Dataset):
    
    def __init__(self, transform=None):
        self.folders = ['wbcs/', 'rbcs/', 'platelets/', 'background/']
        self.inverse_transform = {'background':0, 'rbcs':1, 'wbcs':2, 'platelets':3}
        self.root = 'Patches/'
        self.files = []
        for folder in self.folders:
            temp = os.listdir(self.root + folder)
            for i in range(len(temp)):
                self.files.append(self.root + folder + temp[i])
        self.transform = transform
        
    def __len__(self):
        return len(self.files)
    
    def __getitem__(self, idx):
        img_name = self.files[idx]
        img_file = io.imread(img_name)
        
        if img_file is not None:
            img_file = np.array(img_file, dtype=np.float32)
            img_file /= 255.
            
            if self.transform:
                img_file = self.transform(img_file)
                
        sample = {'image': img_file, 'label': self.inverse_transform[img_name.split('/')[1]]}
        
        return sample

In [5]:
transformed_dataset = CellsDataset(transform=transforms.ToTensor())

In [6]:
data_train_loader = DataLoader(transformed_dataset, batch_size = 64, shuffle=True)

In [7]:
model = LeNet()

In [8]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters())

In [9]:
def train(epoch):
    model.train()
    loss_list, batch_list = [], []
    for i, sample in enumerate(data_train_loader):
        
        image = sample['image']
        label = sample['label']
        image, label = Variable(image), Variable(label)
        
        optimizer.zero_grad()
        output = model(image)
        
        loss = criterion(output, label)
        
        loss_list.append(loss.data[0])
        batch_list.append(i+1)
        
        if i % 100 == 0:
            print('Train - Epoch: %d, Batch: %d, Loss: %f' % (epoch, i, loss.data[0]))
        
        loss.backward()
        optimizer.step()
        
#         _, predicted = output.max(1)
#         correct = predicted.eq(label).sum().item()
#         print('correct = ' + str(correct / 64.0 * 100.))

In [23]:
for e in range(50):
    train(e)

/usr/local/lib/python2.7/dist-packages/ipykernel_launcher.py:37: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
/usr/local/lib/python2.7/dist-packages/ipykernel_launcher.py:15: UserWarning: invalid index of a 0-dim tensor. This will be an error in PyTorch 0.5. Use tensor.item() to convert a 0-dim tensor to a Python number
  from ipykernel import kernelapp as app
/usr/local/lib/python2.7/dist-packages/ipykernel_launcher.py:19: UserWarning: invalid index of a 0-dim tensor. This will be an error in PyTorch 0.5. Use tensor.item() to convert a 0-dim tensor to a Python number


Train - Epoch: 0, Batch: 0, Loss: 1.385581
Train - Epoch: 0, Batch: 100, Loss: 1.174902
Train - Epoch: 0, Batch: 200, Loss: 1.121642
Train - Epoch: 0, Batch: 300, Loss: 1.200180
Train - Epoch: 0, Batch: 400, Loss: 1.080833
Train - Epoch: 0, Batch: 500, Loss: 1.097698
Train - Epoch: 0, Batch: 600, Loss: 1.108816
Train - Epoch: 0, Batch: 700, Loss: 1.085970
Train - Epoch: 0, Batch: 800, Loss: 1.195457
Train - Epoch: 0, Batch: 900, Loss: 1.057411
Train - Epoch: 0, Batch: 1000, Loss: 1.148726
Train - Epoch: 0, Batch: 1100, Loss: 1.064498
Train - Epoch: 0, Batch: 1200, Loss: 1.036080
Train - Epoch: 1, Batch: 0, Loss: 1.162360
Train - Epoch: 1, Batch: 100, Loss: 1.083358
Train - Epoch: 1, Batch: 200, Loss: 1.188177
Train - Epoch: 1, Batch: 300, Loss: 1.156106
Train - Epoch: 1, Batch: 400, Loss: 1.032188
Train - Epoch: 1, Batch: 500, Loss: 1.073159
Train - Epoch: 1, Batch: 600, Loss: 1.007075
Train - Epoch: 1, Batch: 700, Loss: 0.946754
Train - Epoch: 1, Batch: 800, Loss: 1.155814
Train - Epo

KeyboardInterrupt: 

In [11]:
torch.save(model, 'model')

/usr/local/lib/python2.7/dist-packages/torch/serialization.py:241: UserWarning: Couldn't retrieve source code for container of type LeNet. It won't be checked for correctness upon loading.
  "type " + obj.__name__ + ". It won't be checked "


In [16]:
torch.save(model.state_dict(), 'model_dict')

In [19]:
# model1 = LeNet()

In [20]:
# model1.load_state_dict(torch.load('model_dict'))